In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time 

In [2]:
url= 'https://www.qiyas.sa/en/Statistics-Data/Schools/Pages/DetResults.aspx'

#Create a new Safari browser object
driver = webdriver.Safari()
driver.implicitly_wait(100)

#Go to the website
driver.get(url)

##Extract data for Capabilities, Boys, Scientific, Private##
    
#Click on 'Capabilities'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[1]/td/div/div[1]/div').click()

#Click on 'Boys'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[2]/td/div/div[1]').click() 

#Click on 'Scientific'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[3]/td/div/div[1]').click() 

#Click on 'Private'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[4]/td/div/div[2]/div').click() 

#Click on 'View the order of all schools'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[1]/div/table/tbody/tr[5]/td/span[1]/label').click() 

#Click on 'Submit'
driver.find_element_by_xpath('/html/body/form/div[3]/div/div/section/div[2]/div/div[1]/article/div[2]/div[1]/div/div/div/div[1]/div/span/div[2]/div').click()

In [3]:
#Retrieving table from first page 
time.sleep(5)
soup_cbsp= BeautifulSoup(driver.page_source, 'html.parser')

#Parse and extract data 
table= soup_cbsp.find_all(class_='table-responsive')[1]

rows=table.find_all('tr')

COLUMNS= ['General Rank', 'Ministerial Number', 'School Name', 
          'City', 'Region', 'School Type', 'Category', 'Average']

sub_df=[]
for row in rows:
    cells= row.select('td')
    cells=cells[0:8]
    sub_df.append([cell.text.strip() for cell in cells])

data_cbsp = pd.DataFrame(sub_df, columns=COLUMNS).drop([0,1], axis=0).reset_index(drop=True)
data_cbsp


,General Rank,Ministerial Number,School Name,City,Region,School Type,Category,Average
0,4,11175,ثانوية منارات الرياض الأهلية,الرياض,Riyadh,Domestic,مقررات,86.07
1,5,53854,ثانوية الجامعة الأهلية,الخبر,AlSharquia,Domestic,تعليم عام بنين,85.47
2,6,54421,ثانوية اجيال المواهب الاهلية,الخبر,AlSharquia,Domestic,تعليم عام بنين,85.45
3,7,26504,ثانوية الفلاح الأهلية,مكة المكرمة,Makkah,Domestic,تعليم عام بنين,85.01
4,11,538485,ثانوية الحصان النموذجية الاهلية بالدمام - مقررات,الدمام,AlSharquia,Domestic,مقررات,83.65
5,12,53936,ثانوية الانصار الأهلية,الدمام,AlSharquia,Domestic,تعليم عام بنين,83.59
6,14,53853,ثانوية الظهران الاهلية - مقررات,الخبر,AlSharquia,Domestic,مقررات,83.58
7,16,111903,ثانوية الرواد الأهلية - نظام المقررات,الرياض,Riyadh,Domestic,مقررات,83.23
8,18,405625,مدارس منارات المدينة المنورة الأهلية - المرحلة...,المدينة المنورة,Madina,Domestic,مقررات,83.1
9,20,153006,ثانوية المشكاة الأهلية,الدمام,AlSharquia,Domestic,تعليم عام بنين,83.03


In [4]:
while True: 
    try: 
        next_page_btn= driver.find_elements(By.CLASS_NAME, "Next")
        if not next_page_btn:
            print("No next button")
            break
        next_btn= next_page_btn[0]
        if next_btn.get_attribute("class")== "aspNetDisabled Next":
            print("No more pages left")
            break
        else: 
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "Next"))).click()
        
            time.sleep(10)
        
            soup_pgs= BeautifulSoup(driver.page_source, 'html.parser')
        
            #Parse and extract data 
        
            table_pgs= soup_pgs.find_all(class_='table-responsive')[1]
        
            rows_pgs=table_pgs.find_all('tr')
        
            COLUMNS_pgs= ['General Rank', 'Ministerial Number', 'School Name', 
                  'City', 'Region', 'School Type', 'Category', 'Average']
        
            sub_df_pgs=[]
            for row_pgs in rows_pgs:
                cells_pgs= row_pgs.select('td')
                cells_pgs=cells_pgs[0:8]
                sub_df_pgs.append([cell_pgs.text.strip() for cell_pgs in cells_pgs])
        
            data_pgs = pd.DataFrame(sub_df_pgs, columns=COLUMNS_pgs).drop([0,1], axis=0).reset_index(drop=True)
            data_cbsp=data_cbsp.append(data_pgs, ignore_index=True)
        
    except TimeoutException:
        print("Exception has been thrown. No more pages left")
        break
driver.quit()

No more pages left


In [5]:
data_cbsp

,General Rank,Ministerial Number,School Name,City,Region,School Type,Category,Average
0,4,11175,ثانوية منارات الرياض الأهلية,الرياض,Riyadh,Domestic,مقررات,86.07
1,5,53854,ثانوية الجامعة الأهلية,الخبر,AlSharquia,Domestic,تعليم عام بنين,85.47
2,6,54421,ثانوية اجيال المواهب الاهلية,الخبر,AlSharquia,Domestic,تعليم عام بنين,85.45
3,7,26504,ثانوية الفلاح الأهلية,مكة المكرمة,Makkah,Domestic,تعليم عام بنين,85.01
4,11,538485,ثانوية الحصان النموذجية الاهلية بالدمام - مقررات,الدمام,AlSharquia,Domestic,مقررات,83.65
5,12,53936,ثانوية الانصار الأهلية,الدمام,AlSharquia,Domestic,تعليم عام بنين,83.59
6,14,53853,ثانوية الظهران الاهلية - مقررات,الخبر,AlSharquia,Domestic,مقررات,83.58
7,16,111903,ثانوية الرواد الأهلية - نظام المقررات,الرياض,Riyadh,Domestic,مقررات,83.23
8,18,405625,مدارس منارات المدينة المنورة الأهلية - المرحلة...,المدينة المنورة,Madina,Domestic,مقررات,83.1
9,20,153006,ثانوية المشكاة الأهلية,الدمام,AlSharquia,Domestic,تعليم عام بنين,83.03


In [6]:
data_cbsp.isna().sum()

General Rank          0
Ministerial Number    0
School Name           0
City                  0
Region                0
School Type           0
Category              0
Average               0
dtype: int64

In [7]:
data_cbsp.to_csv('cap_boys_sci_priv.csv', index=False)